In [18]:
%matplotlib inline
import matplotlib.pyplot as plt 
from PIL import Image
import torchvision.transforms as T
import cv2
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

# https://haochen23.github.io/2020/04/object-detection-faster-rcnn.html#.Y0VLb8pBxJY

FasterRCNNtest = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features = FasterRCNNtest.roi_heads.box_predictor.cls_score.in_features
FasterRCNNtest.roi_heads.box_predictor = models.detection.faster_rcnn.FastRCNNPredictor(
    in_features, num_classes=8)

torch.save(FasterRCNNtest.state_dict(), f"{'kitti-model'}/model-origin.pth")
FasterRCNNtest.load_state_dict(torch.load('./kitti-model/model4.pth'))

FasterRCNNtest.eval()

#FasterRCNN.eval()

CLASS_NAMES = ['Car', 'Truck', 'Cyclist', 'Tram', 'Person_sitting', 'Misc', 'Van', 'Pedestrian']


def get_prediction(img_path, confidence):
  """
    parameters:
      - img_path - path of the input image
      - confidence - threshold value for prediction score
  """
  img = Image.open(img_path)
  transform = T.Compose([T.ToTensor()])
  img = transform(img)
  pred = FasterRCNNtest([img]) # FasterRCNNtest
  
  print(pred)
  if pred[0]['scores'].detach().numpy().size == 0:
      return None, None
  pred_class = [CLASS_NAMES[i] for i in list(pred[0]['labels'].numpy())]
  pred_boxes = [[(i[0], i[1]), (i[2], i[3])] for i in list(pred[0]['boxes'].detach().numpy())]
  pred_score = list(pred[0]['scores'].detach().numpy())

  for x in pred_score:
    if x > confidence:
      print(pred_score.index(x))
    if pred_score.index(x) == 0:
      return [pred_boxes[0]], [pred_class[0]] 
    
  pred_t = [pred_score.index(x) for x in pred_score if x>confidence][-1]
  pred_boxes = pred_boxes[:pred_t+1]
  pred_class = pred_class[:pred_t+1]
  return pred_boxes, pred_class


def detect_object(img_path, confidence=0.5, rect_th=2, text_size=0.5, text_th=2):
  """
    parameters:
      - img_path - path of the input image
      - confidence - threshold value for prediction score
      - rect_th - thickness of bounding box
      - text_size - size of the class label text
      - text_th - thichness of the text
  """
  boxes, pred_cls = get_prediction(img_path, confidence)
  if boxes == None and pred_cls == None:
    print("NO Detection")
    return None
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  
  for i in range(len(boxes)):
    aa, bb = boxes[i][0][0], boxes[i][0][1]
    a, b = boxes[i][1][0], boxes[i][1][1]
    cv2.rectangle(img, (int(aa), int(bb)), (int(a), int(b)), 
                  color=(0, 255, 0), thickness=rect_th)
    cv2.putText(img, pred_cls[i], (int(aa), int(bb)), cv2.FONT_HERSHEY_SIMPLEX,
                text_size, (0,255,0),thickness=text_th)
  
  plt.figure(figsize=(20,30))
  plt.imshow(img)
  plt.xticks([])
  plt.yticks([])
  plt.show()

detect_object('./kitti-dataset/Kitti/raw/testing/image_2/000423.png', confidence=0.35)

/Applications/anaconda3/envs/tflearn/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/Applications/anaconda3/envs/tflearn/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


[]
NO Detection


In [ ]:
%pip install ipywidgets==7.6
import time
from tqdm import tqdm

for item in tqdm([0, 1, 2, 3, 4]):
    time.sleep(1)

In [ ]:
import numpy as np
a = np.array([[1,1,1,1],[1,1,1,1]])
a = a[:1]
b = np.array([1,1,1,1,1])
b = b[:4]
b